# Import Dataset and Libraries

In [ ]:
%pip install imbalanced-learn==0.10.1
%pip install catboost==1.1.1
%pip install pycountry==22.1.10
%pip install scikit-learn==1.2.2

In [ ]:
import functools

import joblib
import numpy as np
import optuna
import pandas as pd
import pycountry
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (accuracy_score, f1_score, precision_score,
                             recall_score, roc_auc_score)
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler

TRAIN_FILE = "./train.csv"
TEST_FILE = "./submission.csv"
MODEL_FILE = "./catboost_params.pkl"

DO_OPTIMIZATION = False
RANDOM_STATE = 42

In [ ]:
df_train = pd.read_csv(TRAIN_FILE)
df_test = pd.read_csv(TEST_FILE)
df_submission = pd.read_csv(TEST_FILE)

X, y = df_train.drop("is_converted", axis=1), df_train["is_converted"]
X_test = df_test.drop(["is_converted", "id"], axis=1)

# Data Preprocessing

## 1. Data Imputation

In [ ]:
values = {
    **{column: 0 for column in X.columns},
    "inquiry_type": "Quotation or Purchase Consultation",
    "customer_country": "Not Found",
}

X.fillna(value=values, inplace=True)
X_test.fillna(value=values, inplace=True)

## 2. Data Cleaning 

### Feature: Expected Timeline

In [ ]:
def expected_timeline_remap(line):
    if not isinstance(line, str):
        return np.nan

    line = (
        line.lower()
        .strip()
        .replace("then", "than")
        .replace("_", "")
        .replace("~", "")
        .replace(" ", "")
        .replace("-", "")
    )

    if line in ["lessthan3months", "onemonth"]:
        return 1.5
    elif line in ["3months6months", "lessthan6months", "lessthan5months"]:
        return 4.5
    elif line == "6months9months":
        return 7.5
    elif line == "9months1year":
        return 10.5
    elif line == "morethanayear":
        return 12
    else:
        return np.nan


X["expected_timeline"] = [
    expected_timeline_remap(x) for x in X["expected_timeline"].values
]
X_test["expected_timeline"] = [
    expected_timeline_remap(x) for x in X_test["expected_timeline"].values
]

X["expected_timeline"].fillna(X["expected_timeline"].mean(), inplace=True)
X_test["expected_timeline"].fillna(X["expected_timeline"].mean(), inplace=True)

### Feature: Customer Country

In [ ]:
def preprocess_country(country):
    if not isinstance(country, str):
        return "Not Found"

    country = country.split("/")[-1].strip()
    country = pycountry.countries.get(name=country)
    return "Not Found" if country is None else country.alpha_3


X["customer_country"] = [
    preprocess_country(country) for country in X["customer_country"]
]

X_test["customer_country"] = [
    preprocess_country(country) for country in X_test["customer_country"]
]

### Feature: Business Unit ("business_unit")

In [ ]:
X["business_unit"] = X["business_unit"].astype(
    pd.CategoricalDtype(categories=["ID", "AS", "IT", "Solution", "CM"])
)
X_test["business_unit"] = X_test["business_unit"].astype(
    pd.CategoricalDtype(categories=["ID", "AS", "IT", "Solution", "CM"])
)

### Feature: Inquiry Type ("inquiry_type")

In [ ]:
inquiry_type_remap = {
    # Quotation: Asking about the product itself.
    "Q": [
        "Quotation or Purchase Consultation",
        "Quotation or purchase consultation",
        "Quotation or Purchase consultation",
        "quotation_or_purchase_consultation",
        "Purchase or Quotation",
        "Purchase",
        "Request for quotation or purchase",
        "Sales Inquiry",
        "sales",
        "quotation_",
    ],
    # Usage: Asking about how to use the product.
    "U": [
        "Usage or Technical Consultation",
        "Usage or technical consultation",
        "usage or technical consultation",
        "usage_or_technical_consultation",
        "Technical Consultation",
        "Technical Support",
        "technical",
        "technical_consultation",
        "Request for technical consulting",
        "Product Information",
        "i want to know the details about it",
        "first Info and pricing",
        "Toi muon tim hieu thong tin ky thuat, gia ca cua sp de su dung",
        "tôi cần tham khảo giá và giải pháp từ LG",
        "Trainings",
    ],
    "O": [
        "Other",
        "Others",
        "other_",
        "other",
        "others",
        "Etc.",
        "ETC.",
        "Not specified",
        "(Select ID_Needs)",
        "IDB",
        "Services",
        "Request for Partnership",
        "Request a Demo",
        "Request for Distributorship",
        "Customer Suggestions",
        *(
            df_train["inquiry_type"]
            .value_counts()
            .loc[df_train["inquiry_type"].value_counts() < 10]
            .index.tolist()
        ),
    ],
}

for key, value in inquiry_type_remap.items():
    X.loc[X["inquiry_type"].isin(value), "inquiry_type"] = key
    X_test.loc[X_test["inquiry_type"].isin(value), "inquiry_type"] = key

## 3. Feature Engineering

In [ ]:
country_frequency = X["customer_country"].value_counts(normalize=True)
X["customer_country_frequency"] = X["customer_country"].map(country_frequency)
X_test["customer_country_frequency"] = X_test["customer_country"].map(country_frequency)

country_target = (
    pd.concat([X, y], axis=1).groupby("customer_country")["is_converted"].mean()
)
X["customer_country_target"] = X["customer_country"].map(country_target)
X_test["customer_country_target"] = X_test["customer_country"].map(country_target)

owner_frequency = X["lead_owner"].value_counts(normalize=True)
X["lead_owner_frequency"] = X["lead_owner"].map(owner_frequency)
X_test["lead_owner_frequency"] = X_test["lead_owner"].map(owner_frequency)

owner_target = pd.concat([X, y], axis=1).groupby("lead_owner")["is_converted"].mean()
X["lead_owner_target"] = X["lead_owner"].map(owner_target)
X_test["lead_owner_target"] = X_test["lead_owner"].map(owner_target)

business_weight = {
    "ID": 0.064566116,
    "AS": 0.026845638,
    "IT": 0,
    "Solution": 0.034482759,
    "CM": 0,
}

X["com_reg_ver_win_rate_per_bu"] = X["com_reg_ver_win_rate"] * X["business_unit"].map(
    business_weight
)
X_test["com_reg_ver_win_rate_per_bu"] = X_test["com_reg_ver_win_rate"] * X_test[
    "business_unit"
].map(business_weight)

## 4. Feature Encoding

In [ ]:
def encode_features(
    df: pd.DataFrame,
    features=[
        "enterprise",
        "business_unit",
        "inquiry_type",
    ],
) -> pd.DataFrame:
    df_encoded = pd.get_dummies(df[features], columns=features)
    df_encoded = df_encoded.apply(lambda x: x.astype("category").cat.codes)
    df = pd.concat([df, df_encoded], axis=1).drop(features, axis=1)
    return df


X = encode_features(X)
X_test = encode_features(X_test)

## 5. Outlier Removal

In [ ]:
Q1 = X["historical_existing_cnt"].quantile(0.25)
Q3 = X["historical_existing_cnt"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = pd.concat([X, pd.DataFrame({"is_converted": y})], axis=1)
df = df[
    (df["historical_existing_cnt"] >= lower_bound)
    & (df["historical_existing_cnt"] <= upper_bound)
]

## 6. Feature Normalization

In [ ]:
standard_transformer = Pipeline(steps=[("standardize", StandardScaler())])
column_transformer = ColumnTransformer(
    transformers=[
        (
            "std",
            standard_transformer,
            [
                "lead_desc_length",
                "historical_existing_cnt",
                "bant_submit",
                "expected_timeline",
            ],
        ),
    ],
    verbose_feature_names_out=False,
    remainder="passthrough",
)
column_transformer.set_output(transform="pandas")

X = column_transformer.fit_transform(X)
X_test = column_transformer.transform(X_test)

## 7. Feature Removal

In [ ]:
features_to_drop = [
    "customer_country",
    "customer_country.1",
    "business_subarea",
    "business_area",
    "customer_idx",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "customer_job",
    "customer_type",
    "response_corporate",
    "lead_owner",
]

X = X.drop(features_to_drop, axis=1)  # type: ignore
X_test = X_test.drop(features_to_drop, axis=1)  # type: ignore

# Hyperparameter Tuning

In [ ]:
def build_catboost_params(trial):
    return {
        "iterations": trial.suggest_int("cat__iterations", 500, 1500),
        "learning_rate": trial.suggest_float("cat__learning_rate", 1e-3, 0.1),
        "depth": trial.suggest_int("cat__depth", 1, 10),
        "subsample": trial.suggest_float("cat__subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("cat__colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("cat__min_data_in_leaf", 1, 100),
        "verbose": False,
        "random_state": RANDOM_STATE,
        "auto_class_weights": "Balanced",
    }


def optimize(X, y, n_trials) -> None:
    def _optimize(trial, X, y):
        model = CatBoostClassifier(**build_catboost_params(trial))
        kfold, scores = StratifiedKFold(n_splits=5), []
        for train_idx, val_idx in kfold.split(X, y):
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
            model.fit(X_train, y_train)
            prediction = model.predict(X_val)
            prediction = np.where(prediction == "True", True, False)
            scores.append(f1_score(y_val, prediction))
        return np.mean(scores)

    study = optuna.create_study(direction="maximize")
    study.optimize(
        functools.partial(_optimize, X=X, y=y),  # type: ignore
        n_trials=n_trials,
        n_jobs=-1,
        show_progress_bar=True,
    )
    joblib.dump(study, MODEL_FILE)


if DO_OPTIMIZATION:
    optimize(X, y, n_trials=1)

# Modeling

In [ ]:
for col in X.columns:
    print(col)
    print(X[col].head())

param_dict = {
    "cat": {
        "verbose": False,
        "random_state": RANDOM_STATE,
        "auto_class_weights": "Balanced",
    },
}

if DO_OPTIMIZATION:
    optimization_results = joblib.load(MODEL_FILE).best_trial.params
    for key, value in optimization_results.items():
        model, param = [x.strip() for x in key.split("__")]
        param_dict[model][param] = value

model = CatBoostClassifier(**param_dict["cat"])

# K-Fold Cross Validation

In [ ]:
kfolds, scores = (
    StratifiedKFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True),
    [],
)

for fold_idx, (train_idx, val_idx) in enumerate(kfolds.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

    sm = SMOTE(sampling_strategy="minority", random_state=RANDOM_STATE)
    X_train, y_train = sm.fit_resample(X_train, y_train)  # type: ignore

    model.fit(X_train, y_train)
    prediction = model.predict(X_val)
    prediction = np.where(prediction == "True", True, False)

    print(f"Accuracy (Fold {fold_idx}): {accuracy_score(y_val, prediction)}")
    print(f"Precision (Fold {fold_idx}): {precision_score(y_val, prediction)}")
    print(f"Recall (Fold {fold_idx}): {recall_score(y_val, prediction)}")
    print(f"F1-Score (Fold {fold_idx}): {f1_score(y_val, prediction)}")
    print(
        f"ROC-AUC Score (Fold {fold_idx}): "
        + f"{roc_auc_score(y_val, model.predict_proba(X_val)[:,1])}\n"
    )

    scores.append(f1_score(y_val, prediction))

print(f"Cross-Validation Average F1-Score: {np.mean(scores)}")

# Create Submission

In [ ]:
sm = SMOTE(sampling_strategy="minority", random_state=RANDOM_STATE)
X, y = sm.fit_resample(X, y)  # type: ignore
model.fit(X, y)

prediction = model.predict(X_test)
prediction = np.where(prediction == "True", True, False)

df_submission["is_converted"] = prediction
df_submission.to_csv("submission.csv", index=False)
print(df_submission["is_converted"].value_counts())
df_submission.head()